In [0]:
%pip install --quiet mlflow==2.22.0
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.widgets.text("catalog", "mlops")
dbutils.widgets.text("schema", "mlops_zoomcamp_prj")
dbutils.widgets.text("model_name", "software_defects")

model = dbutils.widgets.get("model_name")
# dbutils.widgets.text("experiment_name", "software-defects")

# experiment_name = dbutils.widgets.get("experiment_name")
catalog = dbutils.widgets.get("catalog")
db = dbutils.widgets.get("schema")

In [0]:
import mlflow
from mlflow import MlflowClient
from pyspark.sql.functions import struct, col

mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

In [0]:
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository

requirements_path = ModelsArtifactRepository(f"models:/{catalog}.{db}.{model}@production").download_artifacts(artifact_path="requirements.txt") # download model from remote registry

In [0]:
# %pip install --quiet -r $requirements_path
# dbutils.library.restartPython()


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-r option requires 1 argument


---------------------------------------------------------------------------
CalledProcessError                        Traceback (most recent call last)
File <command-7112813776620554>, line 1
----> 1 get_ipython().run_line_magic('pip', 'install --quiet -r $requirements_path')
      2 dbutils.library.restartPython()

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2480, in InteractiveShell.run_line_magic(self, magic_name, line, _stack_depth)
   2478     kwargs['local_ns'] = self.get_local_scope(stack_depth)
   2479 with self.builtin_trap:
-> 2480     result = fn(*args, **kwargs)
   2482 # The code below prevents the output from being displayed
   2483 # when using magics with decorator @output_can_be_silenced
   2484 # when the last Python token in the expression is a ';'.
   2485 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/PipMagicOverrides.py:82, in PipMagicOverrides.pip(self, line)
     

In [0]:
prediction_columns =   ['loc',
 'v_g',
 'ev_g',
 'iv_g',
 'n',
 'v',
 'l',
 'd',
 'i',
 'e',
 'b',
 't',
 'lOCode',
 'lOComment',
 'lOBlank',
 'locCodeAndComment',
 'uniq_Op',
 'uniq_Opnd',
 'total_Op',
 'total_Opnd',
 'branchCount',
 ]


In [0]:
# Load software features dataset to be scored
inference_df = spark.read.table(f"mlops.mlops_zoomcamp_prj.soft_quality_features")
# Load production  model as a Spark UDF. 
predict_model = mlflow.pyfunc.spark_udf(spark, model_uri=f"models:/{catalog}.{db}.{model}@production") #Use env_manager="virtualenv" to recreate a venv with the same python version if needed

scored_df = inference_df\
    .withColumn('predictions', predict_model(struct(*map(col, prediction_columns))))\
    .drop(col('split'))

display(scored_df)


2025/07/12 11:21:52 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.
2025/07/12 11:21:52 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


loc,v_g,ev_g,iv_g,n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects,predictions
1.1,1.4,1.4,1.4,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.3,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,0,List(0)
1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,List(1)
83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,0.31,1187.7,65.0,10.0,6.0,0.0,18.0,25.0,107.0,64.0,21.0,1,List(1)
46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,0.26,635.37,37.0,2.0,5.0,0.0,16.0,28.0,89.0,52.0,15.0,1,List(1)
25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,0.08,132.33,21.0,0.0,2.0,0.0,11.0,10.0,41.0,17.0,5.0,1,List(1)
43.0,3.0,1.0,3.0,115.0,569.73,0.09,11.27,50.53,6423.73,0.19,356.87,35.0,2.0,4.0,0.0,11.0,20.0,74.0,41.0,5.0,1,List(1)
48.0,6.0,1.0,6.0,149.0,751.61,0.06,15.43,48.72,11596.34,0.25,644.24,41.0,2.0,2.0,0.0,12.0,21.0,95.0,54.0,11.0,1,List(1)
69.0,12.0,1.0,12.0,231.0,1212.27,0.04,27.27,44.45,33061.94,0.4,1836.77,62.0,3.0,2.0,0.0,16.0,22.0,156.0,75.0,23.0,1,List(1)
47.0,6.0,1.0,6.0,149.0,745.0,0.06,16.2,45.99,12069.0,0.25,670.5,41.0,2.0,1.0,0.0,12.0,20.0,95.0,54.0,11.0,1,List(1)
48.0,7.0,1.0,7.0,155.0,801.34,0.06,17.82,44.97,14278.39,0.27,793.24,42.0,2.0,1.0,0.0,14.0,22.0,99.0,56.0,13.0,1,List(1)
